In [ ]:
import os

# Prevent Transformers from trying to import TensorFlow or Flax
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["USE_TF"] = "0"
os.environ["USE_FLAX"] = "0"

import torch
import gradio as gr
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration

# BART large fine-tuned on XSum is optimized for extreme abstractive summarization of news articles,
# making it suitable for generating short, focused summaries from long inputs.
MODEL_NAME = "facebook/bart-large-xsum"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)
model = model.to(device)
model.eval()


def prepare_sample_data():
    dataset = load_dataset("xsum", split="train", streaming=True)
    it = iter(dataset)
    return [next(it)["document"] for _ in range(2)]


@torch.no_grad()
def summarize_news(text: str):
    if not text or len(text.strip()) < 50:
        return "Error: Please provide a longer article (at least 50 characters)."

    try:
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=1024
        ).to(device)

        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=150,
            min_length=50,
            num_beams=4,
            do_sample=False,
            early_stopping=True
        )

        summary = tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )

        return summary

    except Exception as e:
        return f"An error occurred: {str(e)}"


def launch_app():
    sample_articles = prepare_sample_data()

    interface = gr.Interface(
        fn=summarize_news,
        inputs=gr.Textbox(
            lines=12,
            label="Input News Article",
            placeholder="Paste the full text of a news article here..."
        ),
        outputs=gr.Textbox(
            label="Generated Summary",
            interactive=False
        ),
        title="News Summarizer",
        description="This app uses facebook/bart-large-xsum for abstractive summarization.",
        examples=[[s] for s in sample_articles],
    )

    interface.launch(share=True)


if __name__ == "__main__":
    launch_app()


Using device: cuda


c:\Users\Vedant Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Vedant Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://4aee745afe6e753909.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
